In [30]:
from django.db import connection
from django.contrib.auth.models import User
from myboard import models
from django.shortcuts import render

In [5]:
data=User.objects.all()

In [6]:
models.Board??

In [ ]:
#실행햇던 sql history
connection.queries[2]

In [8]:
User.objects.raw("SELECT max(age) FROM myapp_user")

<RawQuerySet: SELECT max(age) FROM myapp_user>

In [9]:
cursor=connection.cursor()
cursor.execute('SELECT max(age) FROM myapp_user')
max_value=cursor.fetchone()[0]

print(max_value)

321


In [90]:
cursor=connection.cursor()
cursor.execute('SELECT *  from myboard_board')
result=cursor.fetchall()

In [12]:
print(type(result))

<class 'list'>


In [14]:
result[0]

IndexError: list index out of range

In [43]:
cursor.execute("select b.id, title, cnt, u.username from myboard_board b , auth_user u where b.author_id = u.id and username='home'")

In [44]:
r=cursor.fetchone()
print(r)

(9, '나는누구인가요', 0, 'home')


In [45]:
result=cursor.fetchall()

In [46]:
for r in result :
    print(r)

(10, '조용한 바람', 0, 'home')
(11, '깊은사랑이죄라면', 0, 'home')
(12, '해피해피해피', 0, 'home')
(13, '해피해피이마트', 0, 'home')
(14, '행복해요 이마트', 0, 'home')
(15, '해피해피해피', 0, 'home')


In [47]:
cursor.description


(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('cnt', None, None, None, None, None, None),
 ('username', None, None, None, None, None, None))

In [50]:
d= dict(zip(['id','title','cnt','username'], result[0]))

print(d)

{'id': 10, 'title': '조용한 바람', 'cnt': 0, 'username': 'home'}


In [61]:
def dictfetchall(cursor):
    desc=cursor.description
    return[
        dict(zip([col[0] for col in desc],row))
        for row in cursor.fetchall()       
    ]

In [150]:
category='common'
name='home'
page=1

cursor=connection.cursor()

cursor.execute(f"""select title, cnt, username 
               from myboard_board b , auth_user u 
               where b.author_id = u.id and username='{name}' and category='{category}'
               """
              )



In [151]:
d = dictfetchall(cursor)

print(d)



[{'title': '나는누구인가요', 'cnt': 1, 'username': 'home'}, {'title': '조용한 바람', 'cnt': 1, 'username': 'home'}, {'title': '깊은사랑이죄라면', 'cnt': 1, 'username': 'home'}, {'title': '해피해피해피', 'cnt': 0, 'username': 'home'}, {'title': '해피해피이마트', 'cnt': 0, 'username': 'home'}, {'title': '행복해요 이마트', 'cnt': 0, 'username': 'home'}, {'title': '해피해피해피', 'cnt': 0, 'username': 'home'}, {'title': 'dfdfasfasdfd', 'cnt': 0, 'username': 'home'}, {'title': 'asfasfasdfasdfas', 'cnt': 0, 'username': 'home'}]


In [155]:
subs=d[0:5]

print(subs)

[{'title': '나는누구인가요', 'cnt': 1, 'username': 'home'}, {'title': '조용한 바람', 'cnt': 1, 'username': 'home'}, {'title': '깊은사랑이죄라면', 'cnt': 1, 'username': 'home'}, {'title': '해피해피해피', 'cnt': 0, 'username': 'home'}, {'title': '해피해피이마트', 'cnt': 0, 'username': 'home'}]


In [10]:
cursor=connection.cursor()

username='home'
sql=f"select id from auth_user where username='{username}'"


cursor.execute(sql)
author_id=cursor.fetchone()[0]
print(author_id)

1


In [18]:
cursor=connection.cursor()
filename='face2.png'

sql=f"""
INSERT INTO myboard_image
("author_id", "filename")
VALUES({author_id},'{filename}')
"""
print(sql)


INSERT INTO myboard_image
("author_id", "filename")
VALUES(1,'face2.png')



In [19]:
v=cursor.execute(sql)

OperationalError: no such table: myboard_image

In [22]:
sql=f"""
CREATE TABLE myboard_image

(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	author_id INTEGER NOT NULL,
	filename VARCHAR(50) NOT NULL,
	FOREIGN KEY (author_id) REFERENCES auth_user(id)
)
"""


In [26]:

username='home'
sql=f"""
    select filename
    from myboard_image 
    where author_id=(select id from auth_user where username='{username}')
    """

print(sql)


    select filename
    from myboard_image 
    where author_id=(select id from auth_user where username='home')
    


In [27]:
cursor=connection.cursor()
cursor.execute(sql)

data=dictfetchall(cursor)
context={"data":data,"username":username}
print(context)

OperationalError: no such table: myboard_image

In [33]:
from django.test import RequestFactory
rf=RequestFactory()


In [31]:
response=render(request ,'myboard/photo.html',context)
print(response.contect.decode())

NameError: name 'request' is not defined